<a href="https://colab.research.google.com/github/clifford-cheng/Lending-Club/blob/master/LendingClub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import os
from tqdm import tqdm


In [2]:
def get_file_list(directory):
    file_list = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.csv'):
            file_list.append(filename)
    return file_list

def p2f(x):
    return float(x.strip('%'))/100

f = lambda x: float(x.strip('%'))/100
s = lambda x: int(x[4:])

def loan(x):
  if x < 10000:
    return 1
  elif 10000 <= x < 20000:
    return 2
  elif 20000 <= x < 30000:
    return 3
  else:
    return 4


def consol_df(directory):
    dataframe = pd.DataFrame(columns = ['id', 'issue_d', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_title', 'emp_length', 'home_ownership',
          'annual_inc', 'verification_status', 'loan_status', 'title', 'addr_state', 'dti', 'delinq_2yrs', 'fico_range_low', 
          'inq_last_6mths', 'pub_rec', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
          'recoveries', 'collection_recovery_fee', 'application_type'])
    df_list = get_file_list(directory)
    for i in tqdm(df_list):    
        df = pd.read_csv('/content/drive/My Drive/Galvanize/Capstones/capstone_3/data/'+ i, header=1, engine='python', 
                            dtype={'id': int, 'desc': str, 'next_pymnt_d': str, 'verification_status_joint':str},
                            skipfooter=4)
        df = df[['id', 'issue_d', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_title', 'emp_length', 'home_ownership',
          'annual_inc', 'verification_status', 'loan_status', 'title', 'addr_state', 'dti', 'delinq_2yrs', 'fico_range_low', 
          'inq_last_6mths', 'pub_rec', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
          'recoveries', 'collection_recovery_fee', 'application_type']]
        df = df[df['loan_status'].isin(('Charged Off', 'Fully Paid'))]
        df = df[df['application_type']=='Individual']

        df['term'] = df['term'].replace(' ', '')
        df['int_rate'] = df['int_rate'].map(f)
        df['issue_d'] = df['issue_d'].map(s)
        df['inq_last_6mths'] = df['inq_last_6mths'].fillna(0).astype(int)
        df['loan_status'] = df['loan_status'].map({'Fully Paid': 1, 'Charged Off': 0})
        df['verification_status'] = df['verification_status'].map({'Verified': 1, 'Source Verified':1, 'Not Verified': 0})
        df = df[df['home_ownership'].isin(['MORTGAGE', 'OWN', 'RENT'])]
        df['loan'] = df['loan_amnt'].map(loan)
        df['emp_length'] = df['emp_length'].replace({'3 years': 3, '10+ years': 10, '4 years': 4, '2 years': 2, '5 years': 5, '6 years': 6,
       '1 year': 1, '7 years': 7, '< 1 year':1, '9 years':9, '8 years':8, None: 0})
        dataframe = pd.concat([dataframe, df], axis = 0)
        df = pd.get_dummies(df, columns=['home_ownership', 'term', 'addr_state'], drop_first=True)

        dataframe = dataframe.reset_index(drop=True)
    return dataframe

In [ ]:
df = consol_df("/content/drive/My Drive/Galvanize/Capstones/capstone_3/data")


 38%|███▊      | 6/16 [02:08<03:20, 20.01s/it]

In [ ]:
df.to_csv('/content/drive/My Drive/Galvanize/Capstones/capstone_3/data/lending_club_data.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Galvanize/Capstones/capstone_3/data/lending_club_data.csv')

In [ ]:

df['loan_status'].corr(df['dti'])


In [ ]:
df['loan_status'].corr(df['fico_range_low'])


In [ ]:
df['loan_status'].corr(df['verification_status'])


In [ ]:
df['loan_status'].corr(df['inq_last_6mths'])

In [ ]:
df['loan_status'].corr(df['emp_length'])

In [ ]:
df['loan_status'].corr(df['annual_inc'])

In [ ]:
df['loan_status'].corr(df['home_ownership_MORTGAGE'])

In [ ]:
df['loan_status'].corr(df['home_ownership_OWN'])

In [ ]:
df['loan_status'].corr(df['home_ownership_RENT'])

In [ ]:
df = pd.get_dummies(df, columns=['home_ownership'], drop_first=True)


In [ ]:
df['loan_status'].corr(df['application_type_Individual'])

In [ ]:
df['loan_status'].corr(df['application_type_Joint App'])

In [ ]:
df

In [ ]:
plot = df.groupby('grade').count().plot.pie(y='id', autopct='%1.1f%%', figsize=(8,8))

In [ ]:
# Success rate of loans by grade.
df.groupby('grade')['loan_status'].sum()/df.groupby('grade')['loan_status'].count() 

In [ ]:
# Interest rate of loans by grade.
df.groupby('grade')['int_rate'].sum()/df.groupby('grade')['int_rate'].count()

In [ ]:
# Average payout of loans by grade.
df.groupby('grade')['loan_status'].sum()/df.groupby('grade')['loan_status'].count() * df.groupby('grade')['int_rate'].sum()/df.groupby('grade')['int_rate'].count()

In [ ]:
# Holy Shit. This means that no matter the grade, all people who can't pay it off do so at the same rate.
# This can be that either frauds are the same or that employment issues/cashflow issues are the same. 
temp_df = df[df['loan_status']==0]
temp_df.groupby('grade')['total_pymnt_inv'].sum()/temp_df.groupby('grade')['loan_amnt'].sum()

In [ ]:
df.groupby(['grade', 'term', 'home_ownership'])['total_pymnt_inv'].sum()/df.groupby(['grade', 'term', 'home_ownership'])['loan_amnt'].sum()

In [ ]:
df.groupby(['grade', 'term'])['total_pymnt_inv'].sum()/df.groupby(['grade', 'term'])['loan_amnt'].sum()

In [ ]:
df.groupby(['grade', 'home_ownership'])['total_pymnt_inv'].sum()/df.groupby(['grade', 'home_ownership'])['loan_amnt'].sum()

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)



df.groupby(['grade', 'loan'])['total_pymnt_inv'].sum()/df.groupby(['grade', 'loan'])['loan_amnt'].sum()